## GAN (Pytorch)

### Terminal : tensorboard --logdir=./GAN

Reference : https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/Pytorch/GANs/1.%20SimpleGAN/fc_gan.py

$$
\underset{\theta_{g}}min \underset{\theta_{d}}max[E_{x\sim P_{data}}logD_{\theta_{d}}(x) + E_{z\sim P_{z}}log(1-D_{\theta_{d}}(G_{\theta_{g}}(z)))]
$$


- For D, maximize objective by making 𝑫(𝒙) is close to 1 and 𝑫(𝑮(𝒛)) is close to 0
- For G, minimize objective by making 𝑫(𝑮(𝒛))

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings('ignore')

In [2]:
class Disciminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128,1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        return self.disc(x)
    
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh(),
        )
    
    def forward(self, x):
        return self.gen(x)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 3e-4
z_dim = 64 #128, 256
image_dim = 28*28*1 #784
batch_size = 32
num_epochs = 10

disc = Disciminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)

fixed_noise = torch.randn((batch_size, z_dim)).to(device)

transforms = transforms.Compose(
[transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))]
)

In [4]:
dataset = datasets.MNIST(root='./content',
                        transform=transforms,
                        download=False)

In [5]:
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
opt_disc = optim.Adam(disc.parameters(),lr=lr)
opt_gen = optim.Adam(gen.parameters(),lr=lr)

In [7]:
criterion = nn.BCELoss()

In [8]:
writer_fake = SummaryWriter(f"./GAN")
writer_real = SummaryWriter(f"./GAN")

In [9]:
step = 0

In [10]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]
        
        ### Train Disciminator : max log(D(real)) + log(1-D(G(z)))
        
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.ones_like(disc_fake))
        
        lossD = (lossD_real + lossD_fake) / 2
        
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()
        
        ### Train Generator min log(1-D(G(z))) <-> max log(D(G(z)))
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        
        if batch_idx == 0:
            print(f"Epoch [{epoch}/{num_epochs}] \ "
                  f"Loss D : {lossD : .4f}, Loss G : {lossG : .4f}"
                 )
            
            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1,1,28,28)
                data = real.reshape(-1,1,28,28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)
                
                writer_fake.add_image(
                "MNIST Fake Images", img_grid_fake, global_step=step)
                
                writer_real.add_image(
                "MNIST Real Images", img_grid_real, global_step=step)
                
                step += 1

Epoch [0/10] \ Loss D :  0.6312, Loss G :  0.6577
Epoch [1/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [2/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [3/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [4/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [5/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [6/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [7/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [8/10] \ Loss D :  0.0000, Loss G :  0.0000
Epoch [9/10] \ Loss D :  0.0000, Loss G :  0.0000


+ cuda device
+ change learning rate
+ change Normalization 
+ change batchnorm
+ architecture change CNN